<img src=https://raw.githubusercontent.com/autonomio/hyperio/master/logo.png>

## How to recover best model from experiment log?
Due to system error or other reason where scan_object is no longer available, it's still possible to get best model/s using nothing but the experiment log. In the below notebook you will learn exactly how.



In [0]:
##!pip install talos

In [17]:
!pip install git+https://github.com/autonomio/talos

  Cloning https://github.com/autonomio/talos to /tmp/pip-req-build-gsk4l970
  Running command git clone -q https://github.com/autonomio/talos /tmp/pip-req-build-gsk4l970
  Created wheel for talos: filename=talos-0.6.4-cp36-none-any.whl size=53460 sha256=9d5e6c36024b39f1bdca65a5482573b28f027a2e97d51f1983273ccf281ddcb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-xjytnr18/wheels/20/f1/9a/63a4168fd779c183fcc5a8ebe6aa34f1f9bc33eaa558e5461b
Successfully built talos


In [0]:
import keras
keras.__version__
import os

In [0]:
import sys
sys.path.insert(0, './talos/')

import sys
sys.path.insert(0, './wrangle/')

import talos
import wrangle
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

First we'll have to perform the `Scan()` experiment to produce the experiment log. Because the experiment log is stored on local machine, interrupted `Scan()` or other reason will not affect its availability. The experiment log is updated after each permutation; it contains an up-to-date record of the experiment.

In [0]:
# load the data
x, y = talos.templates.datasets.iris()
x_train, y_train, x_val, y_val = wrangle.array_split(x, y, .3)

# set the parameter space boundary
p = {'activation':['relu', 'elu'],
     'optimizer': ['Nadam', 'Adam'],
     'losses': ['logcosh'],
     'shapes': ['brick'],
     'first_neuron': [16, 32, 64, 128],
     'hidden_layers':[0, 1, 2, 3],
     'dropout': [.2, .3, .4],
     'batch_size': [20, 30, 40, 50],
     'epochs': [10]}

# define the input model
def iris_model(x_train, y_train, x_val, y_val, params):
    print("\nParams:",params)
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=4, activation=params['activation']))

    talos.utils.hidden_layers(model, params, 3)

    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['acc'])
    # simple early stopping
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    #out = model.fit(x_train, y_train, callbacks=[talos.utils.ExperimentLogCallback('minimal_iris', params),es],
    #                 batch_size=params['batch_size'],
    #                 epochs=params['epochs'],
    #                 validation_data=[x_val, y_val],
    #                 verbose=0)
    out = model.fit(x_train, y_train, callbacks=[es],
                     batch_size=params['batch_size'],
                     epochs=params['epochs'],
                     validation_data=[x_val, y_val],
                     verbose=0)

    return out, model


In [26]:
# start the experiment
scan_object = talos.Scan(x=x_train,
                         y=y_train,
                         x_val=x_val,
                         y_val=y_val,
                         model=iris_model,
                         experiment_name='reactivate',
                         params=p,
                         round_limit=10)



  0%|          | 0/10 [00:00<?, ?it/s]


Params: {'activation': 'relu', 'batch_size': 50, 'dropout': 0.3, 'epochs': 10, 'first_neuron': 64, 'hidden_layers': 1, 'losses': 'logcosh', 'optimizer': 'Adam', 'shapes': 'brick'}




 10%|█         | 1/10 [00:03<00:34,  3.82s/it]


Params: {'activation': 'relu', 'batch_size': 20, 'dropout': 0.4, 'epochs': 10, 'first_neuron': 128, 'hidden_layers': 0, 'losses': 'logcosh', 'optimizer': 'Nadam', 'shapes': 'brick'}




 20%|██        | 2/10 [00:04<00:24,  3.00s/it]

...



 80%|████████  | 8/10 [00:13<00:03,  1.72s/it]


Params: {'activation': 'elu', 'batch_size': 20, 'dropout': 0.2, 'epochs': 10, 'first_neuron': 128, 'hidden_layers': 3, 'losses': 'logcosh', 'optimizer': 'Adam', 'shapes': 'brick'}




 90%|█████████ | 9/10 [00:15<00:01,  1.78s/it]


Params: {'activation': 'relu', 'batch_size': 30, 'dropout': 0.2, 'epochs': 10, 'first_neuron': 32, 'hidden_layers': 3, 'losses': 'logcosh', 'optimizer': 'Adam', 'shapes': 'brick'}




100%|██████████| 10/10 [00:17<00:00,  1.77s/it]



Now we can assume the case where we no longer have access to the `scan_object`. In this `Scan(...experiment_name...)` was set to "reactivate" so we'll find a folder with that name in the present working directory. Next we have to find out what is the name of the experiment log.

In [27]:
# get the name of the experiment log
!ls -lhtr reactivate

total 12K
-rw-r--r-- 1 root root    0 Oct 26 10:09 102619100917.csv
-rw-r--r-- 1 root root    0 Oct 26 10:11 102619101134.csv
-rw-r--r-- 1 root root    0 Oct 26 10:12 102619101222.csv
-rw-r--r-- 1 root root    0 Oct 26 10:44 102619104409.csv
-rw-r--r-- 1 root root    0 Oct 26 10:44 102619104429.csv
-rw-r--r-- 1 root root 1.4K Oct 26 11:00 102619105948.csv
-rw-r--r-- 1 root root 1.3K Oct 26 11:06 102619110642.csv
-rw-r--r-- 1 root root    0 Oct 26 11:16 102619111611.csv
-rw-r--r-- 1 root root    0 Oct 26 11:18 102619111848.csv
-rw-r--r-- 1 root root 1.3K Oct 26 11:20 102619111951.csv


In this case it will be the most recent one so let's go ahead and recover the best models.

In [32]:
from talos.utils.recover_best_model import recover_best_model
results, models = recover_best_model(x_train=x_train,
                                     y_train=y_train,
                                     x_val=x_val,
                                     y_val=y_val,
                                     experiment_log='reactivate/102619111951.csv',
                                     input_model=iris_model,
                                     n_models=5,
                                     task='multi_label')


Params: {'round_epochs': 8, 'val_loss': 0.06707238405942917, 'loss': 0.06938264625413078, 'acc': 0.6571428640967324, 'activation': 'relu', 'batch_size': 50, 'dropout': 0.3, 'epochs': 10, 'first_neuron': 64, 'hidden_layers': 0, 'losses': 'logcosh', 'optimizer': 'Nadam', 'shapes': 'brick'}

Params: {'round_epochs': 10, 'val_loss': 0.002698742396508654, 'loss': 0.026778477288427808, 'acc': 0.9047618962469556, 'activation': 'elu', 'batch_size': 20, 'dropout': 0.2, 'epochs': 10, 'first_neuron': 128, 'hidden_layers': 3, 'losses': 'logcosh', 'optimizer': 'Adam', 'shapes': 'brick'}
Epoch 00009: early stopping

Params: {'round_epochs': 10, 'val_loss': 0.08850399404764175, 'loss': 0.0969679281115532, 'acc': 0.5428571445601327, 'activation': 'relu', 'batch_size': 30, 'dropout': 0.2, 'epochs': 10, 'first_neuron': 32, 'hidden_layers': 3, 'losses': 'logcosh', 'optimizer': 'Adam', 'shapes': 'brick'}

Params: {'round_epochs': 10, 'val_loss': 0.07756999135017395, 'loss': 0.1005504833800452, 'acc': 0.4

Now we can access the cross-validation results:

In [33]:
results

,round_epochs,val_loss,val_acc,loss,acc,activation,batch_size,dropout,epochs,first_neuron,hidden_layers,losses,optimizer,shapes,crossval_mean_f1score
3,8,0.067072,1.000000,0.069383,0.657143,relu,50,0.3,10,64,0,logcosh,Nadam,brick,0.570707
8,10,0.002699,1.000000,0.026778,0.904762,elu,20,0.2,10,128,3,logcosh,Adam,brick,0.872755
9,10,0.088504,0.711111,0.096968,0.542857,relu,30,0.2,10,32,3,logcosh,Adam,brick,0.519192
0,10,0.077570,0.688889,0.100550,0.447619,relu,50,0.3,10,64,1,logcosh,Adam,brick,0.698538
2,10,0.051704,0.688889,0.080007,0.619048,elu,50,0.4,10,64,3,logcosh,Adam,brick,0.157642


We can also access the models and make predictions with them:

In [30]:
models[0].predict(x_val)

array([[0.04286824, 0.43803665, 0.5190951 ],
       [0.83532643, 0.12342866, 0.04124486],
       [0.06288624, 0.4523648 , 0.48474896],
       [0.0801563 , 0.46524096, 0.45460275],
       [0.11095096, 0.49887037, 0.39017868],
       [0.0645155 , 0.46935022, 0.46613425],
       [0.17285007, 0.49693844, 0.3302115 ],
       [0.7595359 , 0.17704529, 0.06341884],
       [0.11147946, 0.5208024 , 0.36771816],
       [0.12562305, 0.50102574, 0.3733512 ],
       [0.05610342, 0.5033231 , 0.4405735 ],
       [0.04560551, 0.4476945 , 0.5067    ],
       [0.0415327 , 0.4665454 , 0.49192187],
       [0.81658334, 0.13574529, 0.04767134],
       [0.84850895, 0.11308668, 0.03840438],
       [0.02503254, 0.44279793, 0.5321696 ],
       [0.03638352, 0.4849997 , 0.47861683],
       [0.14421396, 0.5347575 , 0.32102856],
       [0.10008337, 0.52583647, 0.3740802 ],
       [0.15797704, 0.5135887 , 0.32843426],
       [0.06786589, 0.49733654, 0.43479759],
       [0.03038298, 0.4539205 , 0.5156965 ],
       [0.